# GRU  + ElMO



In [1]:
import pandas as pd
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id")
df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id")

text_train, y_train = df_train["text"].values, df_train["HS"].values
text_dev, y_dev = df_dev["text"].values, df_dev["HS"].values

Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.sequence import pad_sequences


max_length = 30

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

def preprocess_tweet(tweet):
    tokens = tokenizer.tokenize(tweet)
    
    if len(tokens) >= max_length:
        tokens = tokens[:max_length]
    else:
        tokens = tokens + [''] * (max_length - len(tokens))
    return tokens


text_train = [preprocess_tweet(tweet) for tweet in df_train["text"].values]
text_dev = [preprocess_tweet(tweet) for tweet in df_dev["text"].values]

Using TensorFlow backend.


In [3]:
%%capture
from elmoformanylangs import Embedder

e = Embedder("../../../models/elmo/es/")

Carguemos embeddings

In [4]:
import numpy as np

print(text_train[0])


X_train = np.array(e.sents2elmo(text_train))
X_dev = np.array(e.sents2elmo(text_dev))

['easyjet', 'quiere', 'duplicar', 'el', 'número', 'de', 'mujeres', 'piloto', "'", 'verás', 'tú', 'para', 'aparcar', 'el', 'avión', '..', 'http://t.co/46NuLkm09x', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [5]:
X_train.shape, X_dev.shape

((4469, 30, 1024), (500, 30, 1024))

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, Dropout, LSTM
from keras.optimizers import Adam
from keras.preprocessing import sequence

embedding_dim = 1024

optimizer_args = {
    "lr": 0.0005,
    "decay": 0.01
}

model = Sequential()
model.add(GRU(256, input_shape=(max_length, embedding_dim)))
model.add(Dropout(0.75))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 256)               983808    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 1,049,857
Trainable params: 1,049,857
Non-trainable params: 0
_________________________________________________________________
None
Train on 4469 samples, validate on 500 samples
Epoch 1/20
4469/4469 [==============================] - 7s 2ms/step - loss: 0.78

In [7]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 0s 559us/step
Loss        : 0.4448
Accuracy    : 0.7920
Precision   : 0.8073
Recall      : 0.6982
F1          : 0.7488


## Bidirectional GRU

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, Dropout, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam


embedding_dim = 1024

model = Sequential()
model.add(Bidirectional(GRU(256, input_shape=(max_length, embedding_dim))))
model.add(Dropout(0.75))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])



In [9]:

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=25, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/25
4469/4469 [==============================] - 11s 2ms/step - loss: 0.7846 - acc: 0.5659 - val_loss: 0.6508 - val_acc: 0.6280
Epoch 2/25
4469/4469 [==============================] - 9s 2ms/step - loss: 0.6780 - acc: 0.6212 - val_loss: 0.6331 - val_acc: 0.6480
Epoch 3/25
4469/4469 [==============================] - 9s 2ms/step - loss: 0.6264 - acc: 0.6599 - val_loss: 0.6091 - val_acc: 0.6840
Epoch 4/25
4469/4469 [==============================] - 10s 2ms/step - loss: 0.5968 - acc: 0.6894 - val_loss: 0.5853 - val_acc: 0.6920
Epoch 5/25
4469/4469 [==============================] - 10s 2ms/step - loss: 0.5863 - acc: 0.6928 - val_loss: 0.5657 - val_acc: 0.7180
Epoch 6/25
4469/4469 [==============================] - 10s 2ms/step - loss: 0.5453 - acc: 0.7288 - val_loss: 0.5359 - val_acc: 0.7500
Epoch 7/25
4469/4469 [==============================] - 10s 2ms/step - loss: 0.5095 - acc: 0.7545 - val_loss: 0.5120 - val_acc: 0.7540
Epoch 8/25

In [11]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 1s 1ms/step
Loss        : 0.4446
Accuracy    : 0.7940
Precision   : 0.7692
Recall      : 0.7658
F1          : 0.7675


## Error Analysis

Vamos a ver los tweets con mayores errores

In [46]:
df_dev["proba"] = model.predict_proba(X_dev)

true_positives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] >= 0.5)].copy()
true_negatives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] < 0.5)].copy()

false_positives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] > 0.5)].copy()
false_positives.sort_values("proba", ascending=False, inplace=True)


false_negatives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] < 0.5)].copy()
false_negatives.sort_values("proba", ascending=True, inplace=True)




In [55]:
conf_matrix = pd.DataFrame([
    {"real":"hs=1", "pred_true": len(true_positives), "pred_false": len(false_negatives)},
    {"real":"hs=0", "pred_true": len(false_positives), "pred_false": len(true_negatives)}
])

conf_matrix.set_index("real", inplace=True)

conf_matrix[["pred_true", "pred_false"]]

,pred_true,pred_false
real,,
hs=1,170,52
hs=0,51,227


In [56]:
false_negatives.iloc[:10]


,text,HS,TR,AG,proba
id,,,,,
21152,@NatyBurgos15 @Horaciogenta @PaolaPa05 Cállate vos Black and bitch como la puta falsa abogada. Espero se pudra en la cárcel y devuelva todo. Dedícate a estudiar en serio. La dr...,1,1,1,0.018355
21535,#VOX critica duramente a ⁦@pablocasado_⁩ “Les da la bienvenida como si fuera un cartel del welcome de #Carmena “ ¡Expulsión inmediata de moromierdas! https://t.co/HaySDibj2l,1,0,1,0.026838
23415,Exacto! #puralata #feminazis #HombreAbusado https://t.co/zTEvaftxcd,1,0,1,0.067498
24488,Un indocumentado...habla y habla y no para y acaba hablando su ignorancia.Empezamos bien. https://t.co/SMRsqjtEBT,1,0,0,0.068196
24590,"SOMOS MUCHO MÁS DE LO QUE SUEÑAS SER PINCHÉ SUDACA CAGADO, y si no te gusta dime donde nos topamos pa reventarnos el hocico 1:1 @amarchesin1 https://t.co/vHHPBKS1AK",1,1,1,0.076638
24481,En que se parece superman a una mujer lista: En que los dos son personajes de ciencia-ficción.,1,0,0,0.083606
22065,| lo único que dominas es mi polla en tu cara. https://t.co/qlYOd4zPwW,1,1,1,0.083916
20019,"Y NADIE SE HA PREGUNTADO LO QUE LE VA A COSTAR AL HOMBRE DEL GUANTAZO LA SITUACION..?!? PORQUE SEGURO ES, QUE EL MENDA MUSULMONO LE VA A PONER UNA DENUNCIA, QUE EL FALLO VA A S...",1,0,1,0.096642
24352,"¿Ves que tu coño me importe? Cómete mi polla, basura https://t.co/ht02KyYWZh",1,1,1,0.097788


In [58]:
false_positives.sample(n=15)

,text,HS,TR,AG,proba
id,,,,,
22014,Ni tú eres una zorra hambrienta ni yo un incorregible depredador.,0,0,0,0.868541
22889,@elajidetuvida .El acoso y la violación tiene connotaciones muy prifundas. El poder económico de estos acosadores tiene el mismo efecto que un cuchillo en el cuello por violado...,0,0,0,0.624661
24159,"@LeonDeJapon Jajajajajaja tu eres uno, diego jajajajajaja mentira perra perra perrraaaaaaaaaa de mierda jajaja ♡",0,0,0,0.962730
24209,"¿Ya les he contado que si fuera rica/millonaria sería toda una PERRA?, Yisus me está enseñando humildad... Y no he aprendido mucho ldvdd.",0,0,0,0.712699
22938,Masculinidad frágil es mi crush francés echándose un clavado al cenote solo porque un grupo de árabes le dijo pussy,0,0,0,0.749029
23021,Creo que los colegios que impartan el Islam debe de ser el mismo numero de co!egiosque en los países árabes imparten el cristianismo.Que no somos más tontos porque no ensayamo...,0,0,0,0.761440
22517,"-¿eres poblana? -soy colombiana -¿y por qué eres güera? en Colombia TODAS son morenas, fui a Colombia 3 veces. -ok, mentí 🙄 solo me pinto el cabello",0,0,0,0.974465
23380,Pero es que pones Egipto o Emiratos Árabes que son nuestros casos e igual juran que vamos a Qatar gracias a Korea... https://t.co/fJScL7bYfL,0,0,0,0.538016
24066,"👞La huella dejada por los árabes en Al-Andalus, es decir, en España https://t.co/HBoW2AOBey https://t.co/sYejNvzv6j",0,0,0,0.685159
